# 02 Huia Experience Training

### Reference
https://www.tensorflow.org/alpha/tutorials/load_data/images

## TODOS
- not using variable learning rates => keras extension --- 
- LRFinder
- interpret layers visually
- tensorboard convergence

# Setup 
## Install FastAI

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate

In [3]:
bs = 64
import pathlib
from pathlib import Path
#import tensorboard as tb

# Data
We have to understand well our data as this is fundamental to achieve good results.

In [4]:
root_path = pathlib.Path("./training_data/")
image_path = root_path /"images"
json_path = root_path /"json"
image_path,json_path

(PosixPath('training_data/images'), PosixPath('training_data/json'))

In [5]:
# lets set the random seed so we can reproduce our results
import random
random.seed(7)
# get filenames from directories
all_image_paths = list(image_path.glob('*.png'))

#all_image_paths = [path for path in all_image_paths] # convert to strings
#random.shuffle(all_image_paths) # randomize

In [6]:
type(all_image_paths)
#len(all_image_paths)

list

In [7]:
all_image_paths[:20]

[PosixPath('training_data/images/normal_811.png'),
 PosixPath('training_data/images/dramatic_297.png'),
 PosixPath('training_data/images/wings_425.png'),
 PosixPath('training_data/images/wings_417.png'),
 PosixPath('training_data/images/hadouken_1237.png'),
 PosixPath('training_data/images/wings_220.png'),
 PosixPath('training_data/images/hadouken_967.png'),
 PosixPath('training_data/images/wings_37.png'),
 PosixPath('training_data/images/moonwalk_74.png'),
 PosixPath('training_data/images/moonwalk_250.png'),
 PosixPath('training_data/images/dramatic_1256.png'),
 PosixPath('training_data/images/hadouken_262.png'),
 PosixPath('training_data/images/dramatic_555.png'),
 PosixPath('training_data/images/normal_208.png'),
 PosixPath('training_data/images/underarm_460.png'),
 PosixPath('training_data/images/normal_815.png'),
 PosixPath('training_data/images/wings_423.png'),
 PosixPath('training_data/images/backpack_526.png'),
 PosixPath('training_data/images/fly_437.png'),
 PosixPath('trainin

In [8]:
import re
# extract categories for classification
pat = r'/([^/]+)_\d+.png$'
all_image_labels = [str(re.search(pat,str(image)).group(1)).lower() for image in all_image_paths]
all_image_labels

['normal',
 'dramatic',
 'wings',
 'wings',
 'hadouken',
 'wings',
 'hadouken',
 'wings',
 'moonwalk',
 'moonwalk',
 'dramatic',
 'hadouken',
 'dramatic',
 'normal',
 'underarm',
 'normal',
 'wings',
 'backpack',
 'fly',
 'dramatic',
 'dramatic',
 'moonwalk',
 'dramatic',
 'moonwalk',
 'backpack',
 'hadouken',
 'underarm',
 'wings',
 'underarm',
 'hadouken',
 'fly',
 'fly',
 'dramatic',
 'underarm',
 'normal',
 'underarm',
 'dramatic',
 'normal',
 'dramatic',
 'wings',
 'normal',
 'dramatic',
 'wings',
 'normal',
 'wings',
 'backpack',
 'fly',
 'underarm',
 'dramatic',
 'fly',
 'backpack',
 'fly',
 'normal',
 'normal',
 'normal',
 'underarm',
 'dramatic',
 'hadouken',
 'backpack',
 'moonwalk',
 'backpack',
 'moonwalk',
 'moonwalk',
 'moonwalk',
 'fly',
 'moonwalk',
 'dramatic',
 'backpack',
 'dramatic',
 'underarm',
 'moonwalk',
 'hadouken',
 'normal',
 'moonwalk',
 'fly',
 'backpack',
 'moonwalk',
 'dramatic',
 'fly',
 'hadouken',
 'underarm',
 'hadouken',
 'normal',
 'wings',
 'norma

In [9]:
type(models.resnet101)

function

In [10]:
# include transforms we can use (rotate, warp)
# transforms=[rand_resize_crop(224,1.3,(0.85,1.23),jitter(0.01),perspective_warp(),
#                              rotate(),skew()]



data = ImageDataBunch.from_name_re(image_path, all_image_paths, pat, 
                                   ds_tfms=get_transforms
                                   (do_flip=False, 
                                    flip_vert=False, 
                                    max_rotate=4, 
                                    max_zoom=1.1, 
                                    max_lighting=None, 
                                    max_warp=0.2, 
                                    p_affine=0.3, 
                                    p_lighting=0) ,
                                    size=224, bs=bs
                                  ).normalize(imagenet_stats)

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

In [11]:
data.show_batch(rows=3, figsize=(7,6))

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

RuntimeError: Traceback (most recent call last):
  File "/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 41, in _pin_memory_loop
    batch = pin_memory_batch(batch)
  File "/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 58, in pin_memory_batch
    return [pin_memory_batch(sample) for sample in batch]
  File "/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 58, in <listcomp>
    return [pin_memory_batch(sample) for sample in batch]
  File "/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/torch/utils/data/_utils/pin_memory.py", line 50, in pin_memory_batch
    return batch.pin_memory()
RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/THCCachingHostAllocator.cpp:265


In [11]:
print(data.classes)
len(data.classes),data.c

['backpack', 'dramatic', 'fly', 'hadouken', 'moonwalk', 'normal', 'underarm', 'wings']


(8, 8)

In [45]:
# change for mobilenet
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not bool

In [43]:
learn.model
models.resnet34

<function torchvision.models.resnet.resnet34(pretrained=False, **kwargs)>

In [41]:
learn.lr_find()
#learn.recorder.plot()

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

...

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  return torch.gesv(B,A)[0][:,0]
/home/ale/anaconda3/envs/tensor20/lib/python3.6/site-packages/fastai/vision/transform.py:243: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use to

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [ ]:
learn.recorder.plot()

In [ ]:
learn.freeze(:-2)

In [ ]:
learn.fit_one_cycle(20,max_lr=1e-02)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(20, max_lr=slice(1e-8,1e-7))

In [ ]:
learn.save('fastai_huia_pose')
learn.export('fastai_huia_pose')

In [ ]:
import IPython.display as display
import numpy

In [ ]:
#training_data/images/moonwalk_250.png
img = open_image("training_data/images/moonwalk_250.png")
img.show()
learn.predict(img)

#Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))
#doc(plot_top_losses)

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(10,10),dpi=60)

In [ ]:
interp.most_confused(min_val=1)

# Export to Tensorflow

In [24]:
learn = learn.load("fastai_huia_pose")
import torch
from torch.autograd import Variable

dummy_input = Variable(torch.randn(1,3,224,224)).cuda()
torch.onnx.export(learn.model, dummy_input, "huia.onnx")

In [14]:
import onnx
from onnx_tf.backend import prepare
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()



onnx_model = onnx.load("huia.onnx")  # load onnx model
output = prepare(onnx_model).run(input)  # run the loaded model

AttributeError: module 'tensorflow_addons.seq2seq' has no attribute 'hardmax'

In [ ]:
# other converters MMdnn / pytorch2keras

In [ ]:
import tensorflow as tf

import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "models_tfjs/tfjs_huia_mob_224_teste_10_q3",quantization_dtype=np.uint8)

# Tensorboard

In [ ]:
%load_ext tensorboard.notebook

#notebook.list() # View open TensorBoard instances
#notebook.display(port=6006, height=1000)

In [ ]:
#!kill 25264
%tensorboard --logdir ./logs

In [ ]:
import time
default_timeit_steps = 2*steps_per_epoch+1

def timeit(ds, steps=default_timeit_steps):
  overall_start = time.time()
  # Fetch a single batch to prime the pipeline (fill the shuffle buffer),
  # before starting the timer
  it = iter(ds.take(steps+1))
  next(it)

  start = time.time()
  for i,(images,labels) in enumerate(it):
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))
  print("Total time: {}s".format(end-overall_start))

In [ ]:
timeit(ds)

# Predict

In [ ]:
import numpy as np
sample = np.reshape(img_final,[1,224,224,3])

predict = int(model.predict_classes(sample))
predict

In [ ]:
[key for key,value in label_to_index.items() if value == predict]

In [ ]:
label_to_index